In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

!pip install -q keras
import keras
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.models import Sequential,Model
from keras.optimizers import SGD
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Input
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
import os
from keras import regularizers
from keras.optimizers import Adam, SGD, RMSprop
from keras.applications.vgg16 import VGG16,preprocess_input



In [ ]:

from keras.preprocessing.image import ImageDataGenerator
train_data=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_data=ImageDataGenerator(rescale=1./255)
val_data=ImageDataGenerator(rescale=1./255)

train_set=train_data.flow_from_directory('/content/drive/My Drive/chest_xray/train',target_size=(224,224),batch_size=32,class_mode='categorical')
test_set=test_data.flow_from_directory('/content/drive/My Drive/chest_xray/test',target_size=(224,224),batch_size=32,class_mode='categorical')
val_set=val_data.flow_from_directory('/content/drive/My Drive/chest_xray/val',target_size=(224,224),batch_size=32,class_mode='categorical')



In [ ]:

batchX, batchy = train_set.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
x_train_set=train_set.labels 
x_test_set=test_set.labels 


In [ ]:

inputs = Input(shape=(224,224, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=2, activation='sigmoid')(x)

# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')




In [ ]:

def vgg16_model( num_classes=None):

    model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

    x=Dense(1024, activation='relu')(model.layers[-4].output)# add my own dense layer after the last conv block
    x=Dropout(0.7)(x)
    x=Dense(512,activation='relu')(x)
    x=Dropout(0.5)(x)
    x=Dense(2,activation='softmax')(x)
    model=Model(model.input,x)
    
    return model


vgg_conv=vgg16_model(2)
for layer in vgg_conv.layers[:-10]:#freeze all layers except the last ten
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)



In [ ]:
vgg_conv.summary()
opt = Adam(lr=0.0001, decay=1e-5)
early_stop = EarlyStopping(monitor='loss',patience=3,verbose=1)
vgg_conv.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)
path = "/content/drive/My Drive/classifier.h5" 
vgg_conv.save(path)


In [ ]:
hist = vgg_conv.fit_generator(train_set,steps_per_epoch=train_set.samples // 32,epochs=10,verbose=2,validation_data=test_set,validation_steps=test_set.samples//32)


In [ ]:
loss_and_metrics=vgg_conv.evaluate(test_set)
classes=vgg_conv.predict(test_set)
print(loss_and_metrics[0])
print(loss_and_metrics[1])
# print(classes[3])
preds = vgg_conv.predict(val_set)



In [ ]:
print(preds[0][0])
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['acc', 'loss']):
    ax[i].plot(vgg_conv.history[met])
    ax[i].plot(vgg_conv.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])